In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

## Inciso a)

In [2]:
def get_var_cvar(data, positions, conf, long):
    port = data * positions
    port['total'] = port.sum(axis=1)
    port['rendimiento'] = port['total'].pct_change()

    if long == 1:

        var_porcentual = np.percentile(port['rendimiento'].dropna(), 100-conf)
        var_cash = port['total'].iloc[-1] * var_porcentual

        cvar_porcentual = port.query("rendimiento < @var_porcentual")['rendimiento'].mean()
        cvar_cash = port['total'].iloc[-1] * cvar_porcentual

    else:

        var_porcentual = np.percentile(port['rendimiento'].dropna(), conf)
        var_cash = port['total'].iloc[-1] * var_porcentual

        cvar_porcentual = port.query("rendimiento > @var_porcentual")['rendimiento'].mean()
        cvar_cash = port['total'].iloc[-1] * cvar_porcentual

    var_df = pd.DataFrame({
        "Métrica": ["VaR", "cVaR"],
        "Porcentual": [np.round(var_porcentual * 100, 4), np.round(cvar_porcentual * 100, 4)],
        "Cash": [np.round(var_cash, 2), np.round(cvar_cash, 2)]
    })

    return var_df

## Inciso b)

In [3]:
data = yf.download(['CHFMXN=X', 'MXN=X'], start='2020-01-01', end='2024-12-02')['Adj Close'] # DataFrame con los datos de precios históricos
positions = [7100000, 5300000] # Cuanto tienes en cada moneda en orden. Se acomodan en orde alfabético
conf = 99 # Nivel de confianza
long = True # Si es long o short tu posición. Poner True si es long, False si es short

[*********************100%***********************]  2 of 2 completed


In [4]:
get_var_cvar(data, positions, conf, long)

,Métrica,Porcentual,Cash
0,VaR,-1.8482,-5041647.69
1,cVaR,-2.3751,-6478978.99
